In [111]:
import numpy as np

#### Parseo de los elementos del archivo carrays

In [112]:
f = open("conexiontcp.txt", "r") #Abrimos el documento generado
frame = f.readline() #Leemos la primera linea
principal = f.readline() #Leemos la descripción del paquete

nbytes = frame[10]+frame[11] #Obtenemos el número de bytes de la primera línea
nbytes = int(nbytes) #Convertimos el número a entero

ncolumnas = 8 #El número de columnas siempre es 8
nfilas = int(nbytes/ncolumnas) #Dividimos entre el numero de columnas para obtener el número de filas

if (nbytes%8 != 0): #Si el numero de bytes no es multiplo de 8, le añadimos una fila más
    nfilas += 1

#Iniciamos la matriz a none, para que tenga un valor
matriz = [None] * nfilas #Iniciamos la matriz, con el número de filas 
for i in range(nfilas): #Completamos con el número de columnas
    matriz[i] = [None]*ncolumnas


i=0
#Bucle para obtener todos los datos del datagrama
for linea in f:
    j=0 #Inicializamos j al comenzar la fila
    for caracter in linea.split(', '):
        if (j < 8):
            matriz[i][j] = caracter[0:4] #Para que coja solamente los 4 caracteres que queremos (se usa para el último caso sobre todo)
            j+=1 #Incrementamos numero de columnas

    i+=1 #Incrementamos numero de filas
    if(i==nfilas): #Si el numero de filas es igual al limite, paramos la lectura
        break
print (matriz)####CHIVATO
end = f.readline() #Cogemos el }; final
f.readline()#Salto de línea


f.close()

[['0x08', '0x00', '0x27', '0xac', '0xfd', '0x3d', '0x08', '0x00'], ['0x27', '0x15', '0x84', '0x47', '0x08', '0x00', '0x45', '0x00'], ['0x00', '0x3c', '0x7c', '0x51', '0x40', '0x00', '0x40', '0x06'], ['0x3b', '0x0f', '0xc0', '0xa8', '0x01', '0x01', '0xc0', '0xa8'], ['0x01', '0x0a', '0xd2', '0x22', '0x00', '0x50', '0x6c', '0xc8'], ['0xd4', '0x40', '0x00', '0x00', '0x00', '0x00', '0xa0', '0x02'], ['0x72', '0x10', '0x55', '0x73', '0x00', '0x00', '0x02', '0x04'], ['0x05', '0xb4', '0x04', '0x02', '0x08', '0x0a', '0x5c', '0xa1'], ['0x8d', '0x03', '0x00', '0x00', '0x00', '0x00', '0x01', '0x03'], ['0x03', '0x07', None, None, None, None, None, None]]


### Elementos del mensaje

In [113]:
def elementos():
    print ("MAC Destino: ",macdest)
    print ("MAC Origen: ",macorig)
    print ("Tipo MAC: ",tipo)
    print ("VersionIP: ",versionip)
    print ("Cabecera IP: ",cabip)
    print ("Protocolo IP: ",tipoprotocoloip)
    print ("Checksum cabecera IP: ",headerchecksumip)
    print ("IP Destino: ",ipdest)
    print ("IP Origen: ",iporig)
    #print ("Tipo IP: ",iptype)
    #print ("Tipo ICMP: ",icmptype)
    #print ("Código ICMP: ",icmpcode)
    #print ("Checksum ICMP: ",icmpchecksum)
    print("Puerto origen TCP: ",tcpsourceport)
    print("Puerto destino TCP: ",tcpdestport)
    print("Número de secuencia TCP: ",tcpsequencenumber)
    print("Numero de ACK TCP: ",tcpacknowledgenumber)
    print("Longitud de cabecera TCP: ",tcpheaderlenght)
    print("Flag TCP: ",tcpflag)
    print("Tamaño de ventana TCP: ",tcpwinsizevalue)
    print("Checksum TCP: ",tcpchecksum)
    print("Urgent pointer TCP: ",tcpurgentpointer)
    print("Tamaño máximo de segmento TCP: ",tcpmaxsegmentsize)
    print("Opción Sack TCP: ",tcpsackoption)
    print("Timestamps TCP: ",tcptimestamps)
    print("NoOperation TCP: ",tcpnop)
    print("Escala de ventana TCP: ",tcpwindowscale)

### Datagrama ICMP

In [114]:
def protocoloicmp(ipversion,contador, matriz, i,j):
    if ipversion == '4':
        if contador < 35:
            iptype.append(matriz[i][j])
        elif contador < 36:
            icmptype.append(matriz[i][j])
        elif contador < 37:
            icmpcode.append(matriz[i][j])
        elif contador < 39:
            icmpchecksum.append(matriz[i][j])
    elif ipversion== '6':
        if contador < 55:
            iptype.append(matriz[i][j])
        elif contador < 56:
            icmptype.append(matriz[i][j])
        elif contador < 57:
            icmpcode.append(matriz[i][j])
        elif contador < 59:
            icmpchecksum.append(matriz[i][j])

### Flags TCP

In [127]:
def flagstcp(contador, matriz, i, j):
    global flagtcp
    if contador < 50:
        tcpwinsizevalue.append(matriz[i][j]) 
    elif contador < 52:
        tcpchecksum.append(matriz[i][j])
    elif contador < 54:
        tcpurgentpointer.append(matriz[i][j])
    if flagtcp == '02' and contador >= 54:
        if contador < 58:
            tcpmaxsegmentsize.append(matriz[i][j])
        elif contador < 60:
            tcpsackoption.append(matriz[i][j])
        elif contador < 70:
            tcptimestamps.append(matriz[i][j])
        elif contador < 71:
            tcpnop.append(matriz[i][j])
        elif contador < 74:
            tcpwindowscale.append(matriz[i][j])

### Datagrama TCP

In [128]:
def protocolotcp(contador, matriz, i,j):
    global flagtcp
    if contador < 36:
        tcpsourceport.append(matriz[i][j])
    elif contador < 38:
        tcpdestport.append(matriz[i][j])
    elif contador < 42:
        tcpsequencenumber.append(matriz[i][j]) 
    elif contador < 46:
        tcpacknowledgenumber.append(matriz[i][j]) 
    elif contador < 47:
        tcpheaderlenght.append(matriz[i][j])    
    elif contador < 48:
        tcpflag.append(matriz[i][j])
        flagtcp = matriz[i][j][2]+matriz[i][j][3]
    else:
        flagstcp(contador, matriz, i, j)
        

### Datagrama IP

In [129]:
def protocoloip (ipversion,contador, matriz, i,j):
    global ipprotocol
    if ipversion == '4': #Tratamos si usa ipv4
        if contador < 23:
            cabip.append(matriz[i][j])
        elif contador < 24:
            tipoprotocoloip.append(matriz[i][j])
            ipprotocol = matriz[i][j][3]
        elif contador < 26:
            headerchecksumip.append(matriz[i][j])
        elif contador < 30:
            iporig.append(matriz[i][j])
        elif contador < 34:
            ipdest.append(matriz[i][j])
        else:
            if ipprotocol == '6':
                protocolotcp(contador, matriz, i,j)
            else:
                protocoloicmp(ipversion,contador, matriz,i,j) 
    elif ipversion == '6': #Tratamos si usa ipv6
        if contador < 22:
            cabip.append(matriz[i][j])
        elif contador < 38:
            iporig.append(matriz[i][j])
        elif contador < 54:
            ipdest.append(matriz[i][j])
        else:    
            protocoloicmp(contador, matriz,i,j)      

#### Separación de cabeceras y elementos del datagrama

In [130]:
contador = 0 #Contador para ir obteniendo los bytes del mensaje
macdest = [] #Direccion MAC Destino
macorig = [] #Dirección MAC Origen
tipo = [] #Tipo de mensaje que utiliza
versionip = [] #Version de ip utilizada
ipversion = ""

cabip = [] #Parámetros de la cabecera IP
tipoprotocoloip = []#Tipo de protocolo que utiliza el mensaje IP
headerchecksumip = []#Checksum de la cabecera ip
ipdest = [] #Dirección IP Destino
iporig = [] #Dirección IP Origen
ipprotocol = ""

tcpsourceport = [] #Puerto origen para el datagrama TCP
tcpdestport = [] #Puerto destino para el datagrama TCP
tcpsequencenumber = [] #Número de secuencia del datagrama TCP
tcpacknowledgenumber = [] #Número de acknowledge del datagrama TCP
tcpheaderlenght = [] #Longitud de cabecera del datagrama TCP
tcpflag = [] #Flag del datagrama TCP
tcpwinsizevalue = [] #Tamaño de ventana del datagrama TCP
tcpchecksum = [] #Checksum del datagrama TCP
tcpurgentpointer = [] #Urgent pinter del datagrama TCP
tcpmaxsegmentsize = [] #Tamaño máximo del segmento del datagrama TCP
tcpsackoption = [] #Opcion sack del datagrama TCP
tcptimestamps = [] #Timestamos del datagrama TCP
tcpnop = [] #Operacion no del datagrama TCP
tcpwindowscale = [] #Escala de ventana del datagrama TCP
flagtcp = ""

icmptype = [] #Tipo de mensaje ICMP
icmpcode = [] #Código del mesaje ICMP
icmpchecksum = [] #Checksum del ICMP
iptype = [] #Tipo de Mensaje IP

i = 0
#Recorrmos el mesaje extraido y lo vamos separando según los bytes que utiliza
for i in range(nfilas):
    j = 0
    for j in range(ncolumnas):
        if contador < 6:
            macdest.append(matriz[i][j])
        elif contador < 12: 
            macorig.append(matriz[i][j])
        elif contador < 14:  
            tipo.append(matriz[i][j])
        elif contador < 15:
            versionip.append(matriz[i][j])
            ipversion = matriz[i][j][2]
        else:
            protocoloip(ipversion, contador, matriz, i,j)   
        contador += 1
elementos()

MAC Destino:  ['0x08', '0x00', '0x27', '0xac', '0xfd', '0x3d']
MAC Origen:  ['0x08', '0x00', '0x27', '0x15', '0x84', '0x47']
Tipo MAC:  ['0x08', '0x00']
VersionIP:  ['0x45']
Cabecera IP:  ['0x00', '0x00', '0x3c', '0x7c', '0x51', '0x40', '0x00', '0x40']
Protocolo IP:  ['0x06']
Checksum cabecera IP:  ['0x3b', '0x0f']
IP Destino:  ['0xc0', '0xa8', '0x01', '0x0a']
IP Origen:  ['0xc0', '0xa8', '0x01', '0x01']
Puerto origen TCP:  ['0xd2', '0x22']
Puerto destino TCP:  ['0x00', '0x50']
Número de secuencia TCP:  ['0x6c', '0xc8', '0xd4', '0x40']
Numero de ACK TCP:  ['0x00', '0x00', '0x00', '0x00']
Longitud de cabecera TCP:  ['0xa0']
Flag TCP:  ['0x02']
Tamaño de ventana TCP:  ['0x72', '0x10']
Checksum TCP:  ['0x55', '0x73']
Urgent pointer TCP:  ['0x00', '0x00']
Tamaño máximo de segmento TCP:  ['0x02', '0x04', '0x05', '0xb4']
Opción Sack TCP:  ['0x04', '0x02']
Timestamps TCP:  ['0x08', '0x0a', '0x5c', '0xa1', '0x8d', '0x03', '0x00', '0x00', '0x00', '0x00']
NoOperation TCP:  ['0x01']
Escala de ven